In [7]:
import cv2
import zmq
import time
import carla
import pickle
import numpy as np
from threading import Lock

In [10]:
class CarlaServer():
    def __init__(self):
        
        # Carla Connection
        self.client_ = carla.Client("localhost", 2000)
        self.client_.set_timeout(10.0)
        self.world_ = self.client_.get_world()
        self.map_ = self.world_.get_map()
        
        # Spawn Point and start point
        self.spawn_points_ = self.map_.get_spawn_points()
        self.start_point_ = self.spawn_points_[0]
    
        self.blueprint_library_ = self.world_.get_blueprint_library()
        
        # Vehicle Setup
        self.vehicle_bp = self.blueprint_library_.find('vehicle.volkswagen.t2_2021')
        self.vehicle_ = None

        # Camera Setup
        self.camera_bp_ = self.blueprint_library_.find('sensor.camera.rgb')
        self.camera_bp_.set_attribute('image_size_x', '800')
        self.camera_bp_.set_attribute('image_size_y', '288')
        self.camera_bp_.set_attribute('fov', '90')
        
        self.camera_sensor = None
        self.camera_transform = carla.Transform(carla.Location(z=2.5, x=0.65))
        self.image = None
        self.image_lock = Lock()
        
        # GNSS Setup
        self.gnss_lock = Lock()
        self.gnss_bp = self.blueprint_library_.find("sensor.other.gnss")
        self.gnss_transform = carla.Transform(carla.Location(z=2.0))
        self.gnss_sensor = None
        self.gnss_data = {"latitude":0.0, "longitude":0.0}
        
        # XMQ Setup
        context = zmq.Context()
        self.socket = context.socket(zmq.REP)
        self.socket.bind("tcp://*:5550")
        
    def image_callback(self, image):
        array = np.frombuffer(image.raw_data, dtype=np.uint8)
        array = np.reshape(array, (image.height, image.width, 4))[:, :, :3]
        with self.image_lock:
            self.image = array
    
    def gnss_callback(self, event):
        with self.gnss_lock:
            self.gnss_data = {
                "latitude": event.latitude,
                "longitude": event.longitude
            }
        
    def destroy_actors(self):
        if self.camera_sensor is not None:
            self.camera_sensor.stop()
            self.camera_sensor.destroy()
            self.camera_sensor = None

        if self.gnss_sensor is not None:
            self.gnss_sensor.stop()
            self.gnss_sensor.destroy()
            self.gnss_sensor = None

        if self.vehicle_ is not None:
            self.vehicle_.destroy()
            self.vehicle_ = None

                        
    def spawn(self):
        self.destroy_actors()
        self.vehicle_ = self.world_.try_spawn_actor(self.vehicle_bp, self.start_point_)
        
        self.camera_sensor = self.world_.spawn_actor(self.camera_bp_, self.camera_transform, attach_to=self.vehicle_)
        self.camera_sensor.listen(lambda data: self.image_callback(data))
        
        self.gnss_sensor = self.world_.spawn_actor(self.gnss_bp, self.gnss_transform, attach_to=self.vehicle_)
        self.gnss_sensor.listen(lambda data: self.gnss_callback(data))
        print("Vehicle, camera and gnss spawned")
    
    def get_observation(self):
        with self.image_lock:
            if self.image is None:
                raise RuntimeError("Image missing")
            image_copy = self.image.copy()
            
        with self.gnss_lock:
            if self.gnss_data is None:
                raise RuntimeError("GNSS missing")
            gnss_copy = self.gnss_data.copy()

        _, encoded_image = cv2.imencode(".jpg", image_copy)
        return {
            "image": encoded_image.tobytes(),
            "gnss": gnss_copy,
            "timestamp": time.time()
        }


    def run(self):
        self.spawn()
        
        while True:
            message = self.socket.recv()
            try:
                data = pickle.loads(message)
                if data.get("command") == "reset":
                    self.spawn()
                    self.socket.send(pickle.dumps({"status": "reset done"}))
                    continue

                action = data.get("action", [0.0, 0.0])
                steer = float(action[0])
                throttle = 0
                brake = 0
                if action[1] > 0:
                    throttle = action[1]
                else:
                    brake = abs(action[1])
                    
                self.vehicle_.apply_control(
                    carla.VehicleControl(
                        throttle=throttle, 
                        steer=steer, 
                        brake=brake
                    )
                )
                payload = self.get_observation()
                self.socket.send(pickle.dumps(payload))
                
            except Exception as e:
                print(f"[Error h] {e}")
                self.socket.send(pickle.dumps({"error": str(e)}))

In [ ]:
if __name__ == "__main__":
    server = CarlaServer()
    try:
        server.run()
    except KeyboardInterrupt:
        print("Interrupted by user")
    except Exception as e:
        print(f"Fatal Error: {e}")
    finally:
        server.destroy_actors()
        print("Actors destroyed. Goodbye.")

Vehicle, camera and gnss spawned


In [ ]:
         
    #     # Action Space
    #     self.action_space = gym.spaces.Box(
    #         low=np.array([-1.0, -1.0]),   # steer, throttle/brake min values
    #         high=np.array([1.0, 1.0]),    # steer, throttle/brake max values
    #         dtype=np.float32
    #     )
        
    #     # Observation_space
    #     self.observation_space = gym.spaces.Box(
    #         low=0, high=255,
    #         shape=(288, 800, 3),
    #         dtype=np.uint8
    #     )

    
    
    # def reset(self):
    #     self.destroy_actors()
    #     self.vehicle_ = self.world_.try_spawn_actor(self.vehicle_bp, self.start_point_)
    #     self.camera_sensor = self.world_.spawn_actor(
    #         self.camera_bp_,
    #         self.camera_transform, 
    #         attach_to=self.vehicle_
    #     )
    #     self.image = None
    #     self.camera_sensor.listen(lambda data: self.image_callback(data))

    #     while self.image is None:
    #         time.sleep(0.01)

    #     return self.image 
    
    # def step(self, action):
    #     steer = action[0]
    #     throttle = 0
    #     brake = 0
    #     if action[1] > 0:
    #         throttle = action[1]
    #     else:
    #         brake = abs(action[1])
        
    #     self.vehicle_.apply_control(
    #         carla.VehicleControl(
    #             throttle=throttle, 
    #             steer=steer, 
    #             brake=brake
    #         )
    #     )
        
    #     obs = self.image
    #     reward = self.reward()
    #     done = False
    #     info = {}
        
    #     return obs, reward, done, info
    
    # def reward(self):
    # # Example dummy reward — replace with actual logic
    #     return 1.0


In [ ]:
# Lane Detect
# import torch


In [ ]:
# action_space = gym.spaces.Box(
#     low=np.array([-1.0, -0.5]),   # steer, throttle/brake min values
#     high=np.array([1.0, 0.5]),    # steer, throttle/brake max values
#     dtype=np.float32
# )
# for _ in range(5):
#     action = action_space.sample()
#     # print(action)
# print(action_space.sample())

[-0.30524457  0.21000373]
